In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [11]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yinyue_yanzou=pd.DataFrame(columns=columns)
channel_yinyue_yanzou

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').click()  #选择“音乐”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[3]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'音乐'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').click()  #选择“演奏”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').text
sub_channel

'演奏'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [12]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yinyue_yanzou.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-20 08:54:58.861788


<ipython-input-12-d318461b1f2a>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-d318461b1f2a>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-d318461b1f2a>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1rJ41157Gv视频属性的抓取
BV号为1rJ41157Gv的视频属性抓取结束


<ipython-input-12-d318461b1f2a>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为14J411V7Sw视频属性的抓取
BV号为14J411V7Sw的视频属性抓取结束
开始BV号为1mJ411E7jN视频属性的抓取
BV号为1mJ411E7jN的视频属性抓取结束
开始BV号为1uJ411E7wk视频属性的抓取
BV号为1uJ411E7wk的视频属性抓取结束
开始BV号为1EJ41187bv视频属性的抓取
BV号为1EJ41187bv的视频属性抓取结束
开始BV号为1b7411B7Rv视频属性的抓取
BV号为1b7411B7Rv的视频属性抓取结束
开始BV号为1vJ41157bV视频属性的抓取
BV号为1vJ41157bV的视频属性抓取结束
开始BV号为1E7411i7xf视频属性的抓取
BV号为1E7411i7xf的视频属性抓取结束
开始BV号为1U7411k7fx视频属性的抓取
BV号为1U7411k7fx的视频属性抓取结束
开始BV号为14J411L7Zf视频属性的抓取
BV号为14J411L7Zf的视频属性抓取结束
开始BV号为1i7411e746视频属性的抓取
BV号为1i7411e746的视频属性抓取结束
开始BV号为1KJ4117745视频属性的抓取
BV号为1KJ4117745的视频属性抓取结束
开始BV号为197411a7zy视频属性的抓取
BV号为197411a7zy的视频属性抓取结束
开始BV号为1GJ411j7c2视频属性的抓取
BV号为1GJ411j7c2的视频属性抓取结束
开始BV号为11J411p7ss视频属性的抓取
BV号为11J411p7ss的视频属性抓取结束
开始BV号为1UJ411j76M视频属性的抓取
BV号为1UJ411j76M的视频属性抓取结束
开始BV号为1RJ411b7i3视频属性的抓取
BV号为1RJ411b7i3的视频属性抓取结束
开始BV号为1cJ411E78R视频属性的抓取
BV号为1cJ411E78R的视频属性抓取结束
开始BV号为1aJ411p7es视频属性的抓取
BV号为1aJ411p7es的视频属性抓取结束
开始BV号为1yJ411p77g视频属性的抓取
BV号为1yJ411p77g的视频属性抓取结束
开始BV号为1sJ411J7qA视频属性的抓取
BV号为1sJ411J7qA的视频属性抓取结束
开始BV号为1pJ41157mM视频属性的抓取
BV号为1pJ41157mM的视

...

BV号为17V411C7pC的视频属性抓取结束
开始BV号为1aZ4y1p7a8视频属性的抓取
BV号为1aZ4y1p7a8的视频属性抓取结束
开始BV号为1MK411W7Rp视频属性的抓取
BV号为1MK411W7Rp的视频属性抓取结束
开始BV号为1bT4y137uw视频属性的抓取
BV号为1bT4y137uw的视频属性抓取结束
开始BV号为1Az4y197De视频属性的抓取
开始BV号为1Az4y197De视频属性的抓取
BV号为1Az4y197De的视频属性抓取结束
开始BV号为1PZ4y1W7tH视频属性的抓取
BV号为1PZ4y1W7tH的视频属性抓取结束
开始BV号为1wK411p7Aa视频属性的抓取
BV号为1wK411p7Aa的视频属性抓取结束
开始BV号为1qp4y1Q7oD视频属性的抓取
BV号为1qp4y1Q7oD的视频属性抓取结束
开始BV号为1fK4y147x2视频属性的抓取
BV号为1fK4y147x2的视频属性抓取结束
开始BV号为1ef4y1m7Wj视频属性的抓取
BV号为1ef4y1m7Wj的视频属性抓取结束
开始BV号为1Tz4y1d7BR视频属性的抓取
BV号为1Tz4y1d7BR的视频属性抓取结束
开始BV号为1q54y1D7jP视频属性的抓取
BV号为1q54y1D7jP的视频属性抓取结束
开始BV号为1yK411572K视频属性的抓取
BV号为1yK411572K的视频属性抓取结束
开始BV号为1Fg4y1z7U4视频属性的抓取
BV号为1Fg4y1z7U4的视频属性抓取结束
开始BV号为1554y1R7mW视频属性的抓取
开始BV号为1554y1R7mW视频属性的抓取
开始BV号为1554y1R7mW视频属性的抓取
BV号为1554y1R7mW的视频属性抓取结束
开始BV号为1qt4y117nd视频属性的抓取
BV号为1qt4y117nd的视频属性抓取结束
开始BV号为1cg4y1z7rF视频属性的抓取
BV号为1cg4y1z7rF的视频属性抓取结束
开始BV号为1oA41147bB视频属性的抓取
BV号为1oA41147bB的视频属性抓取结束
开始BV号为1F54y1X7NM视频属性的抓取
BV号为1F54y1X7NM的视频属性抓取结束
开始BV号为1JK411p7pd视频属性的抓取
BV号为1JK411p7pd的视

<ipython-input-12-d318461b1f2a>:190: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1xp4y1D79t视频属性的抓取
BV号为1xp4y1D79t的视频属性抓取结束
开始BV号为1k5411W7EX视频属性的抓取
BV号为1k5411W7EX的视频属性抓取结束
开始BV号为13a4y1v7cM视频属性的抓取
BV号为13a4y1v7cM的视频属性抓取结束
开始BV号为1hZ4y1W7RX视频属性的抓取
BV号为1hZ4y1W7RX的视频属性抓取结束
开始BV号为11i4y1G7H6视频属性的抓取
BV号为11i4y1G7H6的视频属性抓取结束
开始BV号为1Tz411v773视频属性的抓取
BV号为1Tz411v773的视频属性抓取结束
开始BV号为1ja4y1v7LW视频属性的抓取
BV号为1ja4y1v7LW的视频属性抓取结束
开始BV号为11K411p7h8视频属性的抓取
BV号为11K411p7h8的视频属性抓取结束
开始BV号为1iT4y1E7vr视频属性的抓取
BV号为1iT4y1E7vr的视频属性抓取结束
开始BV号为12A411q7rz视频属性的抓取
开始BV号为12A411q7rz视频属性的抓取
开始BV号为12A411q7rz视频属性的抓取
BV号为12A411q7rz的视频属性抓取结束
开始BV号为1pK411p7ht视频属性的抓取
BV号为1pK411p7ht的视频属性抓取结束
开始BV号为1KT4y1J7jM视频属性的抓取
BV号为1KT4y1J7jM的视频属性抓取结束
开始BV号为15A411i7YV视频属性的抓取
BV号为15A411i7YV的视频属性抓取结束
开始BV号为1cZ4y1u7B4视频属性的抓取
BV号为1cZ4y1u7B4的视频属性抓取结束
开始BV号为1Qg4y1i7Pa视频属性的抓取
BV号为1Qg4y1i7Pa的视频属性抓取结束
开始BV号为1z54y1B72A视频属性的抓取
BV号为1z54y1B72A的视频属性抓取结束
开始BV号为1GV411k7f4视频属性的抓取
BV号为1GV411k7f4的视频属性抓取结束
开始BV号为1Ft4y1y7Xg视频属性的抓取
BV号为1Ft4y1y7Xg的视频属性抓取结束
开始BV号为1vA411i7SR视频属性的抓取
BV号为1vA411i7SR的视频属性抓取结束
开始BV号为1AC4y1Y77v视频属性的抓取
BV号为1AC4y1Y77v的视

BV号为1564y1c7SC的视频属性抓取结束
开始BV号为1w54y1U7T1视频属性的抓取
BV号为1w54y1U7T1的视频属性抓取结束
开始BV号为1fV41127ki视频属性的抓取
BV号为1fV41127ki的视频属性抓取结束
开始BV号为1p54y1U7xk视频属性的抓取
BV号为1p54y1U7xk的视频属性抓取结束
开始BV号为1i5411h7CQ视频属性的抓取
BV号为1i5411h7CQ的视频属性抓取结束
开始BV号为1F64y1F7AG视频属性的抓取
BV号为1F64y1F7AG的视频属性抓取结束
开始BV号为1jt4y1D714视频属性的抓取
BV号为1jt4y1D714的视频属性抓取结束
开始BV号为1nZ4y1M7gw视频属性的抓取
BV号为1nZ4y1M7gw的视频属性抓取结束
开始BV号为195411a7Qc视频属性的抓取
BV号为195411a7Qc的视频属性抓取结束
开始BV号为1LV411z7ch视频属性的抓取
BV号为1LV411z7ch的视频属性抓取结束
开始BV号为1YV411U78q视频属性的抓取
BV号为1YV411U78q的视频属性抓取结束
开始BV号为14A411n7oS视频属性的抓取
开始BV号为14A411n7oS视频属性的抓取
BV号为14A411n7oS的视频属性抓取结束
开始BV号为1G54y1i7Aw视频属性的抓取
BV号为1G54y1i7Aw的视频属性抓取结束
开始BV号为1dK4y1h7jD视频属性的抓取
BV号为1dK4y1h7jD的视频属性抓取结束
开始BV号为1Si4y1g7cf视频属性的抓取
BV号为1Si4y1g7cf的视频属性抓取结束
开始BV号为1L64y1c7L3视频属性的抓取
BV号为1L64y1c7L3的视频属性抓取结束
开始BV号为18D4y127vT视频属性的抓取
BV号为18D4y127vT的视频属性抓取结束
开始BV号为1CA411n7Yf视频属性的抓取
BV号为1CA411n7Yf的视频属性抓取结束
开始BV号为1i54y127ub视频属性的抓取
BV号为1i54y127ub的视频属性抓取结束
开始BV号为1rA411Y7DZ视频属性的抓取
BV号为1rA411Y7DZ的视频属性抓取结束
开始BV号为1rp4y1Y7um视频属性的抓取
BV号为1rp4y1Y7um的视

开始BV号为1Cy4y1r7Yb视频属性的抓取
BV号为1Cy4y1r7Yb的视频属性抓取结束
开始BV号为1KT4y1F7jx视频属性的抓取
BV号为1KT4y1F7jx的视频属性抓取结束
开始BV号为1Za411w7kD视频属性的抓取
BV号为1Za411w7kD的视频属性抓取结束
开始BV号为16T4y1c7Cy视频属性的抓取
BV号为16T4y1c7Cy的视频属性抓取结束
开始BV号为17K411A7zG视频属性的抓取
BV号为17K411A7zG的视频属性抓取结束
开始BV号为1Py4y187iY视频属性的抓取
BV号为1Py4y187iY的视频属性抓取结束
开始BV号为1xh411973F视频属性的抓取
BV号为1xh411973F的视频属性抓取结束
开始BV号为1DK411A7c2视频属性的抓取
BV号为1DK411A7c2的视频属性抓取结束
开始BV号为1gy4y187LP视频属性的抓取
BV号为1gy4y187LP的视频属性抓取结束
开始BV号为1ka411w7DX视频属性的抓取
BV号为1ka411w7DX的视频属性抓取结束
开始BV号为16z4y1o7dh视频属性的抓取
BV号为16z4y1o7dh的视频属性抓取结束
开始BV号为1Hy4y1r7HD视频属性的抓取
BV号为1Hy4y1r7HD的视频属性抓取结束
开始BV号为18z4y1o7qd视频属性的抓取
BV号为18z4y1o7qd的视频属性抓取结束
开始BV号为1cv411C72d视频属性的抓取
BV号为1cv411C72d的视频属性抓取结束
开始BV号为1JT4y1w7Di视频属性的抓取
BV号为1JT4y1w7Di的视频属性抓取结束
开始BV号为1vK411N7qS视频属性的抓取
BV号为1vK411N7qS的视频属性抓取结束
开始BV号为18T4y1F7Zp视频属性的抓取
BV号为18T4y1F7Zp的视频属性抓取结束
开始BV号为1xa411A7h2视频属性的抓取
开始BV号为1xa411A7h2视频属性的抓取
BV号为1xa411A7h2的视频属性抓取结束
开始BV号为1hr4y1F7bN视频属性的抓取
BV号为1hr4y1F7bN的视频属性抓取结束
开始BV号为1HZ4y1V7vW视频属性的抓取
BV号为1HZ4y1V7vW的视频属性抓取结束
开始BV号为1rf4y1D76h

In [10]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-10-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [13]:
channel_yinyue_yanzou.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
5月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
6月,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98,98
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [14]:
channel_yinyue_yanzou.to_json('channel_yinyue_yanzou.json')